In [1]:
import pandas as pd

In [2]:
tweets = pd.read_csv('tweets.csv')

## Задание 1
Найдите самые короткие твиты авторов, author_name которых состоит из, как минимум, двух слов. Выведите через пробел: author_name и твит.
hint: min(x, key=len)

**p.s. По возможности старайтесь использовать как можно меньше циклов. Используйте логику векторизации pandas**

In [3]:
tweets1 = tweets.copy()
pd.set_option('display.max_colwidth', None)
# избавляемся от имен, в которых только одно слово
tweets1['author_name'] = tweets1['author_name'].apply(lambda x: str(x).split() )
tweets1 = tweets1[tweets1['author_name'].str.len() >= 2]
tweets1['author_name'] = tweets1['author_name'].str.join(' ')
# добавляем стобец с длинной твита (можно и без него, но хотелось видеть длину) длина по символам, а не словам
tweets1['len_tweet'] = tweets1['tweet_content'].str.len()
tweets1.head()

,author_name,created_at,handle,likes,retweets,tweet_content,len_tweet
20,Matt Woods,2019-07-07 14:13:18,matopher,6396,1560,Everyone should find 3 hobbies:\n\nOne that makes you rich. 💰 \nOne that keeps you fit. 🏋️‍♂️\nOne that makes you smart. 📖\n\nh/t: @nava,130.0
22,Thought Reps,2019-07-07 13:18:34,ThoughtReps,333,59,“You’re not going to get rich renting out your time.” \n\nOne of the many brilliant pieces of wisdom from @naval on the @joerogan podcast. \n\nCheck out more:\n,155.0
25,Naval Ravikant Bot,2019-07-06 20:00:35,NavalBot,3171,683,"""The fundamental delusion - there is something out there that will make me happy and fulfilled forever."" - @nava",112.0
27,Naval Ravikant Bot,2019-07-05 20:00:54,NavalBot,1950,411,"""Unnecessary meetings (and most are) are a mutually-assured-destruction of time. Learning how to avoid them is a prerequisite of doing anything great."" - @nava",159.0
28,Shivam Bhardwaj,2019-07-05 17:57:40,sbhardwaj7529,2140,215,The @naval podcast was such a goldmine that I decided to print the entire transcript in the form of a book.\nAbsolutely lovin it. :),132.0


In [4]:
y = tweets1.groupby(by=['author_name']).apply(lambda x: x[x['len_tweet'] == x['len_tweet'].min()])[['tweet_content', 'len_tweet']]
y.sort_values(by='len_tweet')

tweet_content  \
author_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
modest proposal  5369                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             🤔    
                 5409                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             🔥🔥   
                 5363                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             👍    
Mark Ames        8850                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             🤔    
modest proposal  5328                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             🤔    
...                                                                                                                                                                                                                                                                                                                                                                                            

In [29]:
tweets1['tweet_content'] = tweets1['tweet_content'].astype(str) 
x = tweets1.groupby('author_name').apply(lambda x: min(x['tweet_content'], key=len))
# print(x.to_string())

In [6]:
pd.set_option('display.max_colwidth', 50)

## Задание 2
 Выведите частотный список всех эмодзи, использованных в твитах

In [7]:
from collections import Counter
import emoji

all_tweets = tweets.to_string(columns=['tweet_content'])
emo_count = Counter()
for char in all_tweets:
    if char in emoji.UNICODE_EMOJI:
        emo_count[char] += 1 
        
print(emo_count.most_common(25))

[('✅', 117), ('👇', 101), ('😂', 101), ('🔥', 52), ('🏻', 41), ('🙏', 41), ('🏽', 38), ('👏', 34), ('❤', 32), ('♂', 30), ('🚨', 30), ('🇸', 30), ('🇺', 29), ('🏼', 26), ('👨', 20), ('⚫', 19), ('👩', 18), ('👍', 18), ('💯', 17), ('🏾', 17), ('🤔', 17), ('🙌', 16), ('🆕', 16), ('♀', 15), ('🚀', 15)]


## Задание 3
Используя функцию apply, посчитайте сумму sentiment score всех твитов, используя словари: https://github.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/tree/master/data/opinion-lexicon-English

    1 для позитивных слов
    -1 для негативных

Используйте WordNetLemmatizer

Результат запишите в новую колонку sentiment_score



task 3 edit:

После того, как посчитаете сентимент твиттов, отсортируйте все твитты и:
1. Создайте таблицу самых положительных и отрицательных твиттов (по какому-нибудь перцентилю) 
2. Выведите топ 10 наиболее частотных положительных и отрицательных слов для соответствующих по сентименту твиттов

In [8]:
import re
from collections import defaultdict

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

wn = WordNetLemmatizer()

tweets3 = tweets.copy()
tweets3.drop(columns=['created_at', 'handle', 'likes', 'retweets'], inplace=True)
tweets3.head() 

,author_name,tweet_content
0,Naval,"Unresolved thoughts, prematurely pushed out of..."
1,Naval,The modern mind is overstimulated and the mode...
2,Naval,The Lindy Effect for startups:\n\nThe longer y...
3,Naval,@orangebook_ This was a good tweet.
4,Naval,Social media lowers the cost of raising &amp; ...


In [9]:
def lemmatize(text):

    text = str(text).lower()
    text = re.sub(r'[^\w\s]','', text)
    text = text.split()
    
    # лемматизация с чр-тэгами, работает намного дольше
    tag_map = defaultdict(lambda : wordnet.NOUN)
    tag_map['J'] = wordnet.ADJ
    tag_map['V'] = wordnet.VERB
    tag_map['R'] = wordnet.ADV
    res = [wn.lemmatize(x, tag_map[tag[0]]) for x, tag in pos_tag(text)]
    
    # лемматизация без чр-тэгов
    # res = [wn.lemmatize(x) for x in text]
    
    return ' '.join(res)
print(lemmatize('I went and bought two nice bunnies!'))

i go and buy two nice bunny


In [10]:
tweets3['lemmas'] = tweets3['tweet_content'].apply(lemmatize)
tweets3.head()

,author_name,tweet_content,lemmas
0,Naval,"Unresolved thoughts, prematurely pushed out of...",unresolved thought prematurely push out of the...
1,Naval,The modern mind is overstimulated and the mode...,the modern mind be overstimulated and the mode...
2,Naval,The Lindy Effect for startups:\n\nThe longer y...,the lindy effect for startup the longer you go...
3,Naval,@orangebook_ This was a good tweet.,orangebook_ this be a good tweet
4,Naval,Social media lowers the cost of raising &amp; ...,social medium lower the cost of raise amp join...


In [11]:
with open('negative-words.txt','r') as file:
    negative  = file.read()
    negative = negative.split(';;;\n')[2].split()
with open('positive-words.txt','r') as file:
    positive  = file.read()
    positive = positive.split(';;;\n')[2].split()

In [12]:
def sentiments(lemmas):
    total = 0
    for x in lemmas.split():
        if x in positive:
            total += 1
        if x in negative:
            total -= 1
    return total

my_text = tweets3['lemmas'][80]
print(my_text)
print(sentiments(my_text))

modern society will shame you for earn money shame you for be happy shame you for be raise well shame you for have child and ultimately shame you for exist it isnt just religion that control you by declare you a sinner
-2


In [13]:
tweets3['sentiment_score'] = tweets3['lemmas'].apply(sentiments)
tweets3.head()

,author_name,tweet_content,lemmas,sentiment_score
0,Naval,"Unresolved thoughts, prematurely pushed out of...",unresolved thought prematurely push out of the...,-3
1,Naval,The modern mind is overstimulated and the mode...,the modern mind be overstimulated and the mode...,3
2,Naval,The Lindy Effect for startups:\n\nThe longer y...,the lindy effect for startup the longer you go...,0
3,Naval,@orangebook_ This was a good tweet.,orangebook_ this be a good tweet,1
4,Naval,Social media lowers the cost of raising &amp; ...,social medium lower the cost of raise amp join...,-1


In [28]:
# pd с 5 проц самых положителных и самых отрицательных твитов
percent = tweets3['sentiment_score'].quantile([.05, .95]).values
print(percent[0], percent[1])
neg = tweets3[(tweets3['sentiment_score'] <= percent[0] ) | (tweets3['sentiment_score'] >= percent[1])]
neg.sort_values(by='sentiment_score', inplace=True)
neg.reset_index(drop=True, inplace=True)
neg

-3.0 3.0


c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,author_name,tweet_content,lemmas,sentiment_score
0,Orange Book,Growth only requires the courage to understand...,growth only require the courage to understand ...,-13
1,Sahil Lavingia,Being a founder is hard. Being an early employ...,be a founder be hard be an early employee be h...,-12
2,Wealth Theory ™,When a naive investor who saw no risk suddenly...,when a naive investor who saw no risk suddenly...,-11
3,Krishna,"Unease, anxiety, tension, stress, worry - all ...",unease anxiety tension stress worry all form o...,-11
4,Wealth Theory ™,When a naive investor who saw no risk suddenly...,when a naive investor who saw no risk suddenly...,-11
...,...,...,...,...
4639,Mr. Plenty,Women liking rich men doesn’t make them gold d...,woman like rich men doesnt make them gold digg...,10
4640,Alexander J.A Cortes,Tactical Virtues of Masculinity (ala Jack Dono...,tactical virtue of masculinity ala jack donova...,10
4641,The Ancient Sage,Being kind to others is the most effective way...,be kind to others be the most effective way to...,10
4642,The Ancient Sage,Spiritual awakening is the only real freedom.\...,spiritual awakening be the only real freedom f...,10


In [15]:
all_neg = Counter(negative)
all_pos = Counter(positive)

In [16]:
pos_tweets = tweets3[tweets3['sentiment_score'] > 0].to_string(columns=['lemmas'])
neg_tweets = tweets3[tweets3['sentiment_score'] < 0].to_string(columns=['lemmas'])

In [17]:
pos_c = Counter(pos_tweets.split())
neg_c = Counter(neg_tweets.split())

In [18]:
keys = pos_c.keys() & all_pos.keys()
res = Counter(list(keys))
for key in keys:
    res[key] += pos_c[key]
    
res.most_common(10)

[('good', 1555),
 ('like', 1450),
 ('work', 1233),
 ('great', 825),
 ('best', 688),
 ('right', 654),
 ('love', 621),
 ('well', 512),
 ('free', 410),
 ('enough', 362)]

In [19]:
keys2 = neg_c.keys() & all_neg.keys()
res2 = Counter(list(keys2))
for key in keys2:
    res2[key] += neg_c[key]
    
res2.most_common(10)

[('problem', 602),
 ('bad', 531),
 ('lose', 402),
 ('hard', 394),
 ('fear', 389),
 ('wrong', 286),
 ('lie', 222),
 ('waste', 213),
 ('risk', 210),
 ('lack', 209)]